# Lesson 2: Sequential Chats and Customer Onboarding

## Setup

In [1]:
llm_config={"model": "gpt-3.5-turbo"}

In [3]:
from autogen import ConversableAgent

## Creating the needed agents

In [4]:
onboarding_personal_information_agent = ConversableAgent(
    name="Onboarding Personal Information Agent",
    system_message='''You are a helpful customer onboarding agent,
    you are here to help new customers get started with our product.
    Your job is to gather customer's name and location.
    Do not ask for other information. Return 'TERMINATE' 
    when you have gathered all the information.''',
    llm_config=llm_config,
    code_execution_config=False,
    human_input_mode="NEVER",
)

[autogen.oai.client: 02-10 20:18:46] {129} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


In [24]:
onboarding_topic_preference_agent = ConversableAgent(
    name="Onboarding Topic preference Agent",
    system_message='''You are a helpful customer onboarding agent,
    you are here to help new customers get started with our product.
    Your job is to gather customer's preferences on news topics.
    Do not ask for other information.
    Return 'TERMINATE' when you have gathered all the information.''',
    llm_config=llm_config,
    code_execution_config=False,
    human_input_mode="NEVER",
)

[autogen.oai.client: 02-09 23:22:42] {129} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


In [25]:
customer_engagement_agent = ConversableAgent(
    name="Customer Engagement Agent",
    system_message='''You are a helpful customer service agent
    here to provide fun for the customer based on the user's
    personal information and topic preferences.
    This could include fun facts, jokes, or interesting stories.
    Make sure to make it engaging and fun!
    Return 'TERMINATE' when you are done.''',
    llm_config=llm_config,
    code_execution_config=False,
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: "terminate" in msg.get("content").lower(),
)

[autogen.oai.client: 02-09 23:22:42] {129} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


In [26]:
customer_proxy_agent = ConversableAgent(
    name="customer_proxy_agent",
    llm_config=False,
    code_execution_config=False,
    human_input_mode="ALWAYS",
    is_termination_msg=lambda msg: "terminate" in msg.get("content").lower(),
)

## Creating tasks

Now, you can craft a series of tasks to facilitate the onboarding process.

In [27]:
chats = [
    {
        "sender": onboarding_personal_information_agent,
        "recipient": customer_proxy_agent,
        "message": 
            "Hello, I'm here to help you get started with our product."
            "Could you tell me your name and location?",
        "summary_method": "reflection_with_llm",
        "summary_args": {
            "summary_prompt" : "Return the customer information "
                             "into as JSON object only: "
                             "{'name': '', 'location': ''}",
        },
        "max_turns": 2,
        "clear_history" : True
    },
    {
        "sender": onboarding_topic_preference_agent,
        "recipient": customer_proxy_agent,
        "message": 
                "Great! Could you tell me what topics you are "
                "interested in reading about?",
        "summary_method": "reflection_with_llm",
        "max_turns": 1,
        "clear_history" : False
    },
    {
        "sender": customer_proxy_agent,
        "recipient": customer_engagement_agent,
        "message": "Let's find something fun to read.",
        "max_turns": 1,
        "summary_method": "reflection_with_llm",
    },
]

## Start the onboarding process

**Note**: You might get a slightly different response than what's shown in the video. Feel free to try different inputs, such as name, location, and preferences.

In [28]:
from autogen import initiate_chats

chat_results = initiate_chats(chats)


********************************************************************************
Starting a new chat....

********************************************************************************
Onboarding Personal Information Agent (to customer_proxy_agent):

Hello, I'm here to help you get started with our product.Could you tell me your name and location?

--------------------------------------------------------------------------------
customer_proxy_agent (to Onboarding Personal Information Agent):

Kiru, London

--------------------------------------------------------------------------------
Onboarding Personal Information Agent (to customer_proxy_agent):

Thank you for the information, Kiru. Is there anything else you'd like to share with me?

--------------------------------------------------------------------------------
customer_proxy_agent (to Onboarding Personal Information Agent):

Yes

--------------------------------------------------------------------------------

**************

## Print out the summary

In [29]:
for chat_result in chat_results:
    print(chat_result.summary)
    print("\n")

{'name': 'Kiru', 'location': 'London'}


Sure! The main topic of interest for reading is SpaceX.


The main topic of interest for reading is SpaceX. It is a company founded by Elon Musk in 2002 with the goal of reducing space transportation costs and enabling the colonization of Mars. They have made advancements such as launching the Falcon Heavy rocket and have plans to send humans to Mars.




## Print out the cost

In [30]:
for chat_result in chat_results:
    print(chat_result.cost)
    print("\n")

{'usage_including_cached_inference': {'total_cost': 0.00015000000000000001, 'gpt-3.5-turbo-0125': {'cost': 0.00015000000000000001, 'prompt_tokens': 192, 'completion_tokens': 36, 'total_tokens': 228}}, 'usage_excluding_cached_inference': {'total_cost': 0.00015000000000000001, 'gpt-3.5-turbo-0125': {'cost': 0.00015000000000000001, 'prompt_tokens': 192, 'completion_tokens': 36, 'total_tokens': 228}}}


{'usage_including_cached_inference': {'total_cost': 5.1500000000000005e-05, 'gpt-3.5-turbo-0125': {'cost': 5.1500000000000005e-05, 'prompt_tokens': 64, 'completion_tokens': 13, 'total_tokens': 77}}, 'usage_excluding_cached_inference': {'total_cost': 5.1500000000000005e-05, 'gpt-3.5-turbo-0125': {'cost': 5.1500000000000005e-05, 'prompt_tokens': 64, 'completion_tokens': 13, 'total_tokens': 77}}}


{'usage_including_cached_inference': {'total_cost': 0.0003935, 'gpt-3.5-turbo-0125': {'cost': 0.0003935, 'prompt_tokens': 286, 'completion_tokens': 167, 'total_tokens': 453}}, 'usage_excluding_cache